# Imports

In [42]:
from pathlib import Path
import re

from more_itertools import powerset

import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.formula.api import ols

from constants import DataSplit, Model, Metric, DATASET_SYMBOLS, CONSENSUS_BASELINES

# Script Parameters

In [43]:
data_dir = r'./out'  # Directory containing experiment data to analyze
concat_results = True  # If True, join all files within data_dir (and subdirectories, used with replications). If False, only use <data_dir>/results.csv
alpha = 0.05  # Desired alpha for statistical testing (threshold for ANOVA model reduction)

# Print settings
only_reduced = True  # Only display/print reduced models
latex_output = False  # Print LaTeX instead of DataFrames
combine_anova_latex = True  # Combine full and reduced ANOVA models into one LaTeX table (REQUIRES latex_output = True, OVERRIDES only_reduced)

# Function Definitions

In [44]:
# Pivot table aggregation technique
pivot_agg_func = (lambda x: f'${np.mean(x):.3f} \\pm {np.std(x):.3f}$') if latex_output else [np.mean, np.std]

# These functions are used to clean up the ANOVA and LaTeX printouts
def clean_effect_name(name):
    return ':'.join(map(lambda x: x[0], re.findall(r'"([^"]*)"', name))) if ('"' in name) else name

def fix_ordering(df, level=1):
    return df.reindex(columns=df.columns.reindex([metric.value for metric in Metric], level=level)[0])

def get_style(df):
    return df.style.set_table_styles([
        {'selector': 'toprule', 'props': ':hline;'},
        {'selector': 'midrule', 'props': ':hline\hline;'},
        {'selector': 'bottomrule', 'props': ':hline;'}
    ])

def get_anova_latex(df, name):
    df = df.loc[:, (slice(None), ['Coefficient','PR(>F)'])]
    df = df.rename(index={'Residual/Intercept': 'Intercept'}, columns={'PR(>F)': 'P-value'})
    df = fix_ordering(df, level=0)

    s = get_style(df)
    s.format({(metric, col): f'${{:.{precision}f}}$'
              for metric in Metric
              for col, precision in {'Coefficient':5, 'P-value':3}.items()})
    if combine_anova_latex:
        s.format_index('\\rotatebox[origin=c]{{90}}{{{}}}', level=0)

    col_format = '|c|'*combine_anova_latex + '|l|' + 'rc|'*(len(df.columns)//2)

    latex = s.to_latex(column_format=col_format, caption=name, multicol_align='|c|', position='htbp', position_float='centering')
    latex = latex.replace('$nan$','-') # replace NaNs with dashes
    latex = latex.replace('\\\\\nIntercept', '\\\\ \hdashline\nIntercept') # put hdashline before intercept row
    latex = latex.replace('\hline \hdashline', '\hline') # remove hdashline if placed after hline (occurs when intercept is right under the toprule
    if combine_anova_latex:
        latex = latex.replace('\\\\\n &  &', '\\\\\nModel & Effect &') # index names (letting pandas handle this gets the header wrong)
        latex = latex.replace('\\\\\n\multirow', '\\\\ \hline\hline\n\multirow') # put double hline above reduced model
    else:
        latex = latex.replace('\n & Coefficient', '\nEffect & Coefficient') # index name (letting pandas handle this gets the header wrong)

    return latex

# Data Loading

In [45]:
path = Path(data_dir)

pattern = rf'{"**/*" if concat_results else ""}results.csv'

data = pd.concat([pd.read_csv(filename, index_col=0) for filename in path.glob(pattern)], ignore_index=True)
display(data)

,DecisionTree,RandomForest,LogisticRegression,Linear SVM,KNN,RandomBaseline,ConstantBaseline,PreviousBaseline,ConsensusBaseline,Forex,Bond,Index Futures,Commodities Futures,SPY,Random,Test,Accuracy,Macro F1,Weighted F1,ROC AUC
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,0.709172,0.708243,0.709352,0.780650
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,0.499002,0.496684,0.500502,0.507571
2,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0.872483,0.872108,0.872575,0.957834
3,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,0.497006,0.493096,0.498072,0.496413
4,True,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,0.789197,0.785543,0.788714,0.873329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15295,False,False,False,False,False,False,False,False,True,True,False,True,True,True,False,True,0.495475,0.475140,0.501318,NaN
15296,False,False,False,False,False,False,False,False,True,False,True,True,True,True,False,False,0.506523,0.493364,0.510346,NaN
15297,False,False,False,False,False,False,False,False,True,False,True,True,True,True,False,True,0.503145,0.474699,0.511602,NaN
15298,False,False,False,False,False,False,False,False,True,True,True,True,True,True,False,False,0.506523,0.493364,0.510346,NaN


# Asset Presence Analysis (model-averaged, baseline models and training data excluded)

In [46]:
df = data.loc[data[DataSplit.TEST] & ~data[Model.RANDOM_BASELINE] & ~data[Model.CONSTANT_BASELINE] & ~data[Model.PREVIOUS_BASELINE] & ~data[Model.CONSENSUS_BASELINE]]

pivot = [
    df.loc[~df['Random']]
      .pivot_table(values=[metric for metric in Metric],
                   index=asset_type,
                   aggfunc=pivot_agg_func)
    for asset_type in DATASET_SYMBOLS.keys()
]

pivot += [
    df[~df[[asset_type for asset_type in DATASET_SYMBOLS.keys()] + ['Random']].any(axis=1)]
      .pivot_table(values=[metric for metric in Metric],
                   index='SPY',
                   aggfunc=pivot_agg_func),
    df.loc[df['Random']]
      .pivot_table(values=[metric for metric in Metric],
                   index='Random',
                   aggfunc=pivot_agg_func)
]

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['Asset Type','Presence']) \
          .rename(index={'SPY': 'SPY-Only', 'Random': 'Random Data'}) \
          .reindex([True, False], level=1)

if latex_output:
    print(get_style(pivot).to_latex(column_format='|lc|ccc|', position='htbp', position_float='centering'))
else:
    display(fix_ordering(pivot))

mean                                  \
                              Accuracy  Macro F1 Weighted F1   ROC AUC   
Asset Type          Presence                                             
Forex               True      0.530935  0.476261    0.495917  0.511788   
                    False     0.538855  0.484173    0.503574  0.523090   
Bond                True      0.536813  0.487089    0.506093  0.520721   
                    False     0.532977  0.473345    0.493399  0.514157   
Index Futures       True      0.538151  0.485306    0.504566  0.523909   
                    False     0.531640  0.475128    0.494926  0.510969   
Commodities Futures True      0.539238  0.484114    0.505840  0.519746   
                    False     0.530553  0.476320    0.493651  0.515133   
SPY-Only            True      0.528814  0.465952    0.483449  0.510901   
Random Data         True      0.523741  0.462406    0.479670  0.502412   

                                   std                                  
                              Accuracy  Macro F1 Weighted F1   ROC AUC  
Asset Type          Presence                                            
Forex               True      0.029356  0.045655    0.035661  0.027427  
                    False     0.026918  0.050310    0.039917  0.028217  
Bond                True      0.030167  0.043927    0.035300  0.031320  
                    False     0.026463  0.051212    0.039594  0.024695  
Index Futures       True      0.029168  0.047475    0.038286  0.032037  
                    False     0.027309  0.048384    0.037177  0.022410  
Commodities Futures True      0.031196  0.048229    0.037086  0.032670  
                    False     0.024633  0.047857    0.038009  0.023116  
SPY-Only            True      0.024477  0.044986    0.033837  0.014424  
Random Data         True      0.024065  0.054962    0.043363  0.019051

# Asset Combinations (model-averaged, baseline models and training data excluded)

In [47]:
df = data.loc[data[DataSplit.TEST] & ~data[Model.RANDOM_BASELINE] & ~data[Model.CONSTANT_BASELINE] & ~data[Model.PREVIOUS_BASELINE] & ~data[Model.CONSENSUS_BASELINE]].copy()

for asset_type in DATASET_SYMBOLS.keys():
    df[asset_type] = df[asset_type].map({True: asset_type[0], False: ''})

df['Random'] = df['Random'].map({True: 'Random Data', False: ''})

df['Asset Combination'] = df[[asset_type for asset_type in DATASET_SYMBOLS.keys()] + ['Random']].apply(lambda x: ''.join(x.values.astype(str)), axis=1)

pivot = df.pivot_table(values=[metric for metric in Metric],
                       index='Asset Combination',
                       aggfunc=pivot_agg_func)

pivot = pivot.rename(index={'': 'SPY-Only'})
pivot = pivot.reindex([''.join(c) for c in powerset(''.join(asset_type[0] for asset_type in DATASET_SYMBOLS.keys()))] + ['SPY-Only', 'Random Data'])
pivot = pivot.drop(index='')

if latex_output:
    print(get_style(pivot).to_latex(column_format='|l|ccc|', position='htbp', position_float='centering'))
else:
    display(fix_ordering(pivot))

mean                                       std  \
                   Accuracy  Macro F1 Weighted F1   ROC AUC  Accuracy   
Asset Combination                                                       
F                  0.519840  0.464578    0.480770  0.502201  0.021387   
B                  0.536510  0.487091    0.504226  0.523287  0.021979   
I                  0.534004  0.479261    0.496312  0.520436  0.021749   
C                  0.532031  0.463290    0.489193  0.501274  0.027954   
FB                 0.528649  0.480183    0.497638  0.514243  0.024817   
FI                 0.540070  0.486601    0.503300  0.523089  0.024984   
FC                 0.534858  0.460599    0.487684  0.507550  0.024783   
BI                 0.532933  0.474393    0.493322  0.522191  0.022601   
BC                 0.548795  0.504627    0.522452  0.528001  0.028946   
IC                 0.542434  0.489468    0.509061  0.532033  0.025419   
FBI                0.523600  0.472500    0.490192  0.504713  0.028162   
FBC                0.523620  0.474701    0.493992  0.500292  0.032358   
FIC                0.531767  0.477008    0.497420  0.515776  0.032995   
BIC                0.555319  0.509298    0.530581  0.546598  0.029813   
FBIC               0.545077  0.493918    0.516340  0.526441  0.033776   
SPY-Only           0.528814  0.465952    0.483449  0.510901  0.024477   
Random Data        0.523741  0.462406    0.479670  0.502412  0.024065   

                                                   
                   Macro F1 Weighted F1   ROC AUC  
Asset Combination                                  
F                  0.054932    0.042922  0.013238  
B                  0.045686    0.034775  0.016645  
I                  0.045216    0.036194  0.019405  
C                  0.052168    0.038417  0.019099  
FB                 0.041825    0.032720  0.023977  
FI                 0.051367    0.042973  0.028213  
FC                 0.051716    0.037329  0.020352  
BI                 0.055305    0.044183  0.029510  
BC                 0.039061    0.029889  0.024690  
IC                 0.054482    0.041251  0.026708  
FBI                0.035448    0.026650  0.022519  
FBC                0.037446    0.028127  0.025610  
FIC                0.045645    0.034514  0.032248  
BIC                0.043681    0.034801  0.040956  
FBIC               0.032787    0.023451  0.034885  
SPY-Only           0.044986    0.033837  0.014424  
Random Data        0.054962    0.043363  0.019051

# Model Performance (dataset averaged, random data excluded)
## Out-sample

In [48]:
df = data.loc[~data['Random']]

pivot = [
    df.loc[df[DataSplit.TEST]]
      .pivot_table(values=[metric for metric in Metric],
                   index=model,
                   aggfunc=pivot_agg_func)
    for model in Model
]

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['Model'])
pivot = pivot.loc[pivot.index.get_level_values(1)].droplevel(1)

if latex_output:
    print(get_style(pivot).to_latex(column_format='|l|ccc|', position='htbp', position_float='centering'))
else:
    display(fix_ordering(pivot))

mean                                       std  \
                    Accuracy  Macro F1 Weighted F1   ROC AUC  Accuracy   
Model                                                                    
DecisionTree        0.510665  0.499409    0.508287  0.502165  0.023628   
RandomForest        0.535501  0.488149    0.508631  0.505507  0.020405   
Linear SVM          0.569656  0.400815    0.442721  0.538370  0.011752   
KNN                 0.522114  0.501280    0.513969  0.512971  0.022962   
LogisticRegression  0.536539  0.511431    0.525120  0.528182  0.021049   
RandomBaseline      0.516363  0.512286    0.518188  0.500000  0.025863   
ConstantBaseline    0.566104  0.361456    0.409296  0.500000  0.008061   
PreviousBaseline    0.535565  0.527326    0.535495       NaN  0.020922   
ConsensusBaseline   0.495972  0.474116    0.503530       NaN  0.010374   

                                                    
                    Macro F1 Weighted F1   ROC AUC  
Model                                               
DecisionTree        0.024706    0.023284  0.026341  
RandomForest        0.023350    0.021359  0.023863  
Linear SVM          0.029612    0.028253  0.018295  
KNN                 0.023775    0.022354  0.024241  
LogisticRegression  0.029390    0.026266  0.029966  
RandomBaseline      0.025654    0.025946  0.000000  
ConstantBaseline    0.003284    0.009553  0.000000  
PreviousBaseline    0.020591    0.020893       NaN  
ConsensusBaseline   0.007761    0.011036       NaN

## In-sample

In [49]:
df = data.loc[~data['Random']]

pivot = [
    df.loc[~df[DataSplit.TEST]]
      .pivot_table(values=[metric for metric in Metric],
                   index=model,
                   aggfunc=pivot_agg_func)
    for model in Model
]

pivot = pd.concat(pivot, keys=[tab.index.name for tab in pivot], names=['Model','used'])
pivot = pivot.loc[pivot.index.get_level_values(1)].droplevel(1)

if latex_output:
    print(get_style(pivot).to_latex(column_format='|l|ccc|', position='htbp', position_float='centering'))
else:
    display(fix_ordering(pivot))

mean                                       std  \
                    Accuracy  Macro F1 Weighted F1   ROC AUC  Accuracy   
Model                                                                    
DecisionTree        0.830631  0.827126    0.829314  0.898762  0.099054   
RandomForest        0.998363  0.998331    0.998349  0.999830  0.009840   
Linear SVM          0.571199  0.408890    0.440566  0.675030  0.023494   
KNN                 0.711307  0.704566    0.709061  0.755570  0.112681   
LogisticRegression  0.638896  0.622700    0.629983  0.685808  0.057788   
RandomBaseline      0.501323  0.499650    0.502654  0.500000  0.008180   
ConstantBaseline    0.551458  0.355430    0.392056  0.500000  0.007491   
PreviousBaseline    0.477660  0.471936    0.477672       NaN  0.008782   
ConsensusBaseline   0.507286  0.496815    0.510188       NaN  0.007238   

                                                    
                    Macro F1 Weighted F1   ROC AUC  
Model                                               
DecisionTree        0.103327    0.101483  0.085312  
RandomForest        0.010131    0.010015  0.001848  
Linear SVM          0.046327    0.045119  0.144062  
KNN                 0.114817    0.113229  0.098197  
LogisticRegression  0.067987    0.066298  0.076808  
RandomBaseline      0.008217    0.008108  0.000000  
ConstantBaseline    0.003136    0.008709  0.000000  
PreviousBaseline    0.009732    0.008775       NaN  
ConsensusBaseline   0.006879    0.007447       NaN

# Statistical Analysis of Factor Effects (random data baseline excluded)
## ANOVA Model Generation

In [50]:
all_df = data.loc[data[DataSplit.TEST] & ~data['Random']]
all_df = all_df.replace({True: 1, False: -1}) # required to get coefficients. !does not change results!

anovas = {}

for model in Model:
    df = all_df.loc[data[model]]

    anovas[model] = {
        'full': {},
        'reduced': {}
    }

    if not latex_output:
        print(f'\nAnalyzing {model}...')

    for metric in Metric:
        if model in CONSENSUS_BASELINES and metric is Metric.ROC_AUC:
            continue

        relation = f'Q("{metric}") ~ ' + ' * '.join(['Q("'+asset_type+'")' for asset_type in DATASET_SYMBOLS.keys()])
        glm = ols(relation, data=df).fit()
        aov = sm.stats.anova_lm(glm, typ=1)

        aov = aov.rename(index={'Residual': 'Residual/Intercept'})
        coefs = glm.params.rename(index={'Intercept': 'Residual/Intercept'})
        coefs.name = 'Coefficient'

        anovas[model]['full'][metric.value] = aov.join(coefs)

        if not latex_output:
            print(f'\nReducing over {metric}...')

        no_reductions = True

        # while non-significant effects, remove the least significant effect and associated interactions and refit ANOVA model
        while (aov['PR(>F)'] > alpha).any():
            no_reductions = False
            rem_effect = aov['F'].idxmin()
            if not latex_output:
                print(f'removing effect {clean_effect_name(rem_effect)} (p={aov["PR(>F)"].max():.3f})')
            for effect in aov.index:
                if all(main_effect in effect.split(':') for main_effect in rem_effect.split(':')):
                    relation += f' - {effect}'

            glm = ols(relation, data=df).fit()
            aov = sm.stats.anova_lm(glm, typ=1)

        if no_reductions:
            if not latex_output:
                print('no effects removed')
        else:
            coefs = glm.params.rename(index={'Intercept': 'Residual/Intercept'})
            coefs.name = 'Coefficient'
            aov = aov.rename(index={'Residual': 'Residual/Intercept'})
            anovas[model]['reduced'][metric.value] = aov.join(coefs)


Analyzing DecisionTree...

Reducing over Accuracy...
removing effect B:C (p=0.986)
removing effect F:B:I (p=0.797)
removing effect F (p=0.785)
removing effect C (p=0.087)
removing effect B:I (p=0.076)
removing effect B (p=0.068)

Reducing over Macro F1...
removing effect F (p=0.837)
removing effect B:I (p=0.699)
removing effect B:C (p=0.504)
removing effect I (p=0.258)
removing effect B (p=0.086)

Reducing over Weighted F1...
removing effect B:I (p=0.947)
removing effect F (p=0.764)
removing effect B:C (p=0.720)
removing effect C (p=0.346)
removing effect I (p=0.052)

Reducing over ROC AUC...
removing effect F:B:I (p=0.994)
removing effect B:I:C (p=0.872)
removing effect F:C (p=0.821)
removing effect B:C (p=0.541)
removing effect B (p=0.420)
removing effect I:C (p=0.207)
removing effect F (p=0.074)
removing effect I (p=0.073)

Analyzing RandomForest...

Reducing over Accuracy...
removing effect B:C (p=0.699)
removing effect F:I (p=0.554)
removing effect F:C (p=0.476)
removing effect I

## Printouts

In [51]:
index_ordering = [':'.join(c) for c in powerset(''.join(asset_type[0] for asset_type in DATASET_SYMBOLS.keys()))][1:] + ['Residual/Intercept']

for model, to_join in anovas.items():
    full_join = to_join['full']
    reduced_join = to_join['reduced']

    full_model = pd.concat(full_join, axis=1)
    full_model = full_model.set_index(full_model.index.map(clean_effect_name)).reindex(index_ordering)

    reduced_model = None

    if len(reduced_join) == len(Metric):
        reduced_model = pd.concat(reduced_join, axis=1)
    elif len(reduced_join) > 0:
        to_join = {metric: reduced_join[metric]
                            if metric in reduced_join.keys()
                            else full_join[metric]
                   for metric in full_join.keys()}
        reduced_model = pd.concat(to_join, axis=1)

    if len(reduced_join) == 0: # no reduction occurred
        if latex_output: # print LaTeX
            if combine_anova_latex: # combined table format
                print(get_anova_latex(pd.concat({'Full \& Reduced': full_model}), f'{model}'))
            else: # individual table format
                print(get_anova_latex(full_model, f'{model} (Full \& Reduced)'))
        else: # display DataFrame
            print(f'\n{model} (Full & Reduced):')
            display(full_model)
    else:  # reduction occurred
        reduced_model = reduced_model.set_index(reduced_model.index.map(clean_effect_name)).reindex(index_ordering)
        reduced_model = reduced_model.dropna(how='all')
        if latex_output:  # print LaTeX
            if combine_anova_latex: # combine full and reduced ANOVA models into one table
                print(get_anova_latex(pd.concat({'Full':full_model, 'Reduced':reduced_model}), f'{model}'))
            else:  # individual table format
                if not only_reduced:  # print full model if desired
                    print(get_anova_latex(full_model, f'{model} (Full)'))
                print(get_anova_latex(reduced_model, f'{model} (Reduced)'))
        else:  # display DataFrames
            if not only_reduced:
                print(f'\n{model} (Full):')
                display(full_model)
            print(f'\n{model} (Reduced):')
            display(reduced_model)


DecisionTree (Reduced):


Accuracy                                           \
                         df    sum_sq   mean_sq          F    PR(>F)   
B                       NaN       NaN       NaN        NaN       NaN   
I                       1.0  0.005590  0.005590  10.128264  0.001517   
C                       NaN       NaN       NaN        NaN       NaN   
Residual/Intercept    798.0  0.440466  0.000552        NaN       NaN   

                               Macro F1                                ...  \
                   Coefficient       df    sum_sq   mean_sq         F  ...   
B                          NaN      NaN       NaN       NaN       NaN  ...   
I                     0.002643      NaN       NaN       NaN       NaN  ...   
C                          NaN      1.0  0.003404  0.003404  5.609455  ...   
Residual/Intercept    0.510665    798.0  0.484278  0.000607       NaN  ...   

                   Weighted F1                                 ROC AUC  \
                       mean_sq         F    PR(>F) Coefficient      df   
B                     0.002875  5.331766  0.021195    0.001896     NaN   
I                          NaN       NaN       NaN         NaN     NaN   
C                          NaN       NaN       NaN         NaN     1.0   
Residual/Intercept    0.000539       NaN       NaN    0.508287   798.0   

                                                                       
                      sum_sq   mean_sq         F   PR(>F) Coefficient  
B                        NaN       NaN       NaN      NaN         NaN  
I                        NaN       NaN       NaN      NaN         NaN  
C                   0.004482  0.004482  6.503896  0.01095   -0.002367  
Residual/Intercept  0.549888  0.000689       NaN      NaN    0.502165  

[4 rows x 24 columns]


RandomForest (Reduced):


Accuracy                                               \
                         df    sum_sq   mean_sq          F        PR(>F)   
F                       NaN       NaN       NaN        NaN           NaN   
B                       1.0  0.016696  0.016696  42.655980  1.162122e-10   
I                       NaN       NaN       NaN        NaN           NaN   
C                       1.0  0.004036  0.004036  10.312908  1.374026e-03   
B:I                     NaN       NaN       NaN        NaN           NaN   
B:C                     NaN       NaN       NaN        NaN           NaN   
I:C                     NaN       NaN       NaN        NaN           NaN   
Residual/Intercept    797.0  0.311947  0.000391        NaN           NaN   

                               Macro F1                                 ...  \
                   Coefficient       df    sum_sq   mean_sq          F  ...   
F                          NaN      1.0  0.007890  0.007890  15.188130  ...   
B                     0.004568      NaN       NaN       NaN        NaN  ...   
I                          NaN      1.0  0.006522  0.006522  12.554708  ...   
C                     0.002246      1.0  0.003549  0.003549   6.831312  ...   
B:I                        NaN      NaN       NaN       NaN        NaN  ...   
B:C                        NaN      NaN       NaN       NaN        NaN  ...   
I:C                        NaN      1.0  0.004700  0.004700   9.048528  ...   
Residual/Intercept    0.535501    795.0  0.412972  0.000519        NaN  ...   

                   Weighted F1                                  ROC AUC  \
                       mean_sq          F    PR(>F) Coefficient      df   
F                     0.005429  12.505089  0.000429   -0.002605     1.0   
B                     0.004148   9.554344  0.002064    0.002277     1.0   
I                     0.002975   6.853676  0.009014   -0.001928     1.0   
C                          NaN        NaN       NaN         NaN     1.0   
B:I                   0.006855  15.791801  0.000077   -0.002927     1.0   
B:C                        NaN        NaN       NaN         NaN     1.0   
I:C                        NaN        NaN       NaN         NaN     1.0   
Residual/Intercept    0.000434        NaN       NaN    0.508631   792.0   

                                                                             
                      sum_sq   mean_sq          F        PR(>F) Coefficient  
F                   0.003334  0.003334   6.543518  1.071173e-02   -0.002041  
B                   0.004180  0.004180   8.205684  4.286409e-03    0.002286  
I                   0.004002  0.004002   7.854570  5.193065e-03   -0.002237  
C                   0.004113  0.004113   8.072671  4.609167e-03   -0.002267  
B:I                 0.024380  0.024380  47.854270  9.470471e-12   -0.005520  
B:C                 0.003180  0.003180   6.241273  1.268235e-02    0.001994  
I:C                 0.008307  0.008307  16.305810  5.913801e-05    0.003222  
Residual/Intercept  0.403494  0.000509        NaN           NaN    0.505507  

[8 rows x 24 columns]


Linear SVM (Reduced):


...

Accuracy                                                   \
                         df        sum_sq       mean_sq             F PR(>F)   
F                       1.0  1.568013e-07  1.568013e-07  1.093112e+23    0.0   
B                       1.0  2.584296e-03  2.584296e-03  1.801596e+27    0.0   
I                       1.0  1.363344e-04  1.363344e-04  9.504312e+25    0.0   
C                       1.0  4.335580e-02  4.335580e-02  3.022472e+28    0.0   
F:B                     1.0  1.568013e-07  1.568013e-07  1.093112e+23    0.0   
F:I                     1.0  1.568013e-07  1.568013e-07  1.093112e+23    0.0   
F:C                     1.0  1.568013e-07  1.568013e-07  1.093112e+23    0.0   
B:I                     1.0  1.711348e-03  1.711348e-03  1.193036e+27    0.0   
B:C                     1.0  6.813323e-04  6.813323e-04  4.749786e+26    0.0   
I:C                     1.0  5.512252e-04  5.512252e-04  3.842768e+26    0.0   
F:B:I                   1.0  1.568013e-07  1.568013e-07  1.093112e+23    0.0   
F:B:C                   1.0  1.568013e-07  1.568013e-07  1.093112e+23    0.0   
F:I:C                   1.0  1.568013e-07  1.568013e-07  1.093112e+23    0.0   
B:I:C                   1.0  2.893242e-03  2.893242e-03  2.016972e+27    0.0   
F:B:I:C                 1.0  1.568013e-07  1.568013e-07  1.093112e+23    0.0   
Residual/Intercept    784.0  1.124608e-27  1.434448e-30           NaN    NaN   

                               Macro F1                              \
                   Coefficient       df        sum_sq       mean_sq   
F                     0.000014      1.0  2.674921e-08  2.674921e-08   
B                     0.001797      1.0  4.324960e-04  4.324960e-04   
I                     0.000413      1.0  2.170232e-05  2.170232e-05   
C                     0.007362      1.0  7.204023e-03  7.204023e-03   
F:B                  -0.000014      1.0  2.674921e-08  2.674921e-08   
F:I                  -0.000014      1.0  2.674921e-08  2.674921e-08   
F:C                  -0.000014      1.0  2.674921e-08  2.674921e-08   
B:I                   0.001463      1.0  2.776303e-04  2.776303e-04   
B:C                  -0.000923      1.0  1.180900e-04  1.180900e-04   
I:C                   0.000830      1.0  9.021523e-05  9.021523e-05   
F:B:I                 0.000014      1.0  2.674921e-08  2.674921e-08   
F:B:C                 0.000014      1.0  2.674921e-08  2.674921e-08   
F:I:C                 0.000014      1.0  2.674921e-08  2.674921e-08   
B:I:C                 0.001902      1.0  4.733810e-04  4.733810e-04   
F:B:I:C              -0.000014      1.0  2.674921e-08  2.674921e-08   
Residual/Intercept    0.566104    784.0  3.375339e-28  4.305279e-31   

                                  ...   Weighted F1                       \
                               F  ...       mean_sq             F PR(>F)   
F                   6.213118e+22  ...  2.160940e-07  9.662760e+23    0.0   
B                   1.004571e+27  ...  3.609471e-03  1.613995e+28    0.0   
I                   5.040862e+25  ...  1.969902e-04  8.808525e+26    0.0   
C                   1.673300e+28  ...  6.085501e-02  2.721165e+29    0.0   
F:B                 6.213118e+22  ...  2.160940e-07  9.662760e+23    0.0   
F:I                 6.213118e+22  ...  2.160940e-07  9.662760e+23    0.0   
F:C                 6.213118e+22  ...  2.160940e-07  9.662760e+23    0.0   
B:I                 6.448603e+26  ...  2.441586e-03  1.091769e+28    0.0   
B:C                 2.742911e+26  ...  9.284723e-04  4.151715e+27    0.0   
I:C                 2.095456e+26  ...  7.817634e-04  3.495698e+27    0.0   
F:B:I               6.213118e+22  ...  2.160940e-07  9.662760e+23    0.0   
F:B:C               6.213118e+22  ...  2.160940e-07  9.662760e+23    0.0   
F:I:C               6.213118e+22  ...  2.160940e-07  9.662760e+23    0.0   
B:I:C               1.099536e+27  ...  4.104079e-03  1.835161e+28    0.0   
F:B:I:C             6.213118e+22  ...  2.160940e-07  9.662760e+23    0.0   
Residua


PreviousBaseline (Full & Reduced):


Accuracy                                                   \
                         df        sum_sq       mean_sq             F PR(>F)   
F                       1.0  2.837609e-06  2.837609e-06  6.374850e+23    0.0   
B                       1.0  1.845878e-01  1.845878e-01  4.146871e+28    0.0   
I                       1.0  6.900714e-02  6.900714e-02  1.550285e+28    0.0   
C                       1.0  1.369912e-02  1.369912e-02  3.077586e+27    0.0   
F:B                     1.0  2.837609e-06  2.837609e-06  6.374850e+23    0.0   
F:I                     1.0  2.837609e-06  2.837609e-06  6.374850e+23    0.0   
F:C                     1.0  2.837609e-06  2.837609e-06  6.374850e+23    0.0   
B:I                     1.0  3.938534e-02  3.938534e-02  8.848141e+27    0.0   
B:C                     1.0  1.708752e-02  1.708752e-02  3.838809e+27    0.0   
I:C                     1.0  3.143824e-03  3.143824e-03  7.062779e+26    0.0   
F:B:I                   1.0  2.837609e-06  2.837609e-06  6.374850e+23    0.0   
F:B:C                   1.0  2.837609e-06  2.837609e-06  6.374850e+23    0.0   
F:I:C                   1.0  2.837609e-06  2.837609e-06  6.374850e+23    0.0   
B:I:C                   1.0  2.281427e-02  2.281427e-02  5.125355e+27    0.0   
F:B:I:C                 1.0  2.837609e-06  2.837609e-06  6.374850e+23    0.0   
Residual/Intercept    784.0  3.489785e-27  4.451256e-30           NaN    NaN   

                               Macro F1                              \
                   Coefficient       df        sum_sq       mean_sq   
F                     0.000060      1.0  3.314457e-06  3.314457e-06   
B                     0.015190      1.0  1.801031e-01  1.801031e-01   
I                     0.009288      1.0  6.929678e-02  6.929678e-02   
C                     0.004138      1.0  4.212335e-03  4.212335e-03   
F:B                  -0.000060      1.0  3.314457e-06  3.314457e-06   
F:I                  -0.000060      1.0  3.314457e-06  3.314457e-06   
F:C                  -0.000060      1.0  3.314457e-06  3.314457e-06   
B:I                  -0.007017      1.0  4.536263e-02  4.536263e-02   
B:C                  -0.004622      1.0  1.700520e-02  1.700520e-02   
I:C                  -0.001982      1.0  3.813663e-03  3.813663e-03   
F:B:I                 0.000060      1.0  3.314457e-06  3.314457e-06   
F:B:C                 0.000060      1.0  3.314457e-06  3.314457e-06   
F:I:C                 0.000060      1.0  3.314457e-06  3.314457e-06   
B:I:C                 0.005340      1.0  1.893889e-02  1.893889e-02   
F:B:I:C              -0.000060      1.0  3.314457e-06  3.314457e-06   
Residual/Intercept    0.535565    784.0  3.212075e-27  4.097035e-30   

                                                    Weighted F1                \
                               F PR(>F) Coefficient          df        sum_sq   
F                   8.089893e+23    0.0    0.000064         1.0  3.193616e-06   
B                   4.395938e+28    0.0    0.015004         1.0  1.837626e-01   
I                   1.691389e+28    0.0    0.009307         1.0  6.878781e-02   
C                   1.028142e+27    0.0    0.002295         1.0  1.347498e-02   
F:B                 8.089893e+23    0.0   -0.000064         1.0  3.193616e-06   
F:I                 8.089893e+23    0.0   -0.000064         1.0  3.193616e-06   
F:C                 8.089893e+23    0.0   -0.000064         1.0  3.193616e-06   
B:I                 1.107206e+28    0.0   -0.007530         1.0  3.930574e-02   
B:C                 4.150612e+27    0.0   -0.004610         1.0  1.761178e-02   
I:C                 9.308350e+26    0.0   -0.002183         1.0  3.121364e-03   
F:B:I               8.089893e+23    0.0    0.000064         1.0  3.193616e-06   
F:B:C               8.089893e+23    0.0    0.000064         1.0  3.193616e-06   
F:I:C               8.089893e+23    0.0    0.000064         1.0  3.193616e-06   
B:I:C               4.622584e+27    0.0    0.004866         1.0  2.268823e-02   
F:B


ConsensusBaseline (Full & Reduced):


Accuracy                                                   \
                         df        sum_sq       mean_sq             F PR(>F)   
F                       1.0  2.787305e-06  2.787305e-06  6.579990e+23    0.0   
B                       1.0  1.975174e-03  1.975174e-03  4.662793e+26    0.0   
I                       1.0  1.672096e-03  1.672096e-03  3.947316e+26    0.0   
C                       1.0  5.842379e-02  5.842379e-02  1.379210e+28    0.0   
F:B                     1.0  2.787305e-06  2.787305e-06  6.579990e+23    0.0   
F:I                     1.0  2.787305e-06  2.787305e-06  6.579990e+23    0.0   
F:C                     1.0  2.787305e-06  2.787305e-06  6.579990e+23    0.0   
B:I                     1.0  2.295189e-03  2.295189e-03  5.418253e+26    0.0   
B:C                     1.0  1.382131e-05  1.382131e-05  3.262798e+24    0.0   
I:C                     1.0  1.132571e-02  1.132571e-02  2.673660e+27    0.0   
F:B:I                   1.0  2.787305e-06  2.787305e-06  6.579990e+23    0.0   
F:B:C                   1.0  2.787305e-06  2.787305e-06  6.579990e+23    0.0   
F:I:C                   1.0  2.787305e-06  2.787305e-06  6.579990e+23    0.0   
B:I:C                   1.0  1.025780e-02  1.025780e-02  2.421558e+27    0.0   
F:B:I:C                 1.0  2.787305e-06  2.787305e-06  6.579990e+23    0.0   
Residual/Intercept    784.0  3.321049e-27  4.236032e-30           NaN    NaN   

                               Macro F1                              \
                   Coefficient       df        sum_sq       mean_sq   
F                     0.000059      1.0  3.324723e-06  3.324723e-06   
B                    -0.001571      1.0  8.202611e-03  8.202611e-03   
I                    -0.001446      1.0  9.691294e-04  9.691294e-04   
C                     0.008546      1.0  2.306695e-02  2.306695e-02   
F:B                  -0.000059      1.0  3.324723e-06  3.324723e-06   
F:I                  -0.000059      1.0  3.324723e-06  3.324723e-06   
F:C                  -0.000059      1.0  3.324723e-06  3.324723e-06   
B:I                   0.001694      1.0  4.928827e-05  4.928827e-05   
B:C                   0.000131      1.0  2.181993e-06  2.181993e-06   
I:C                  -0.003763      1.0  9.605932e-03  9.605932e-03   
F:B:I                 0.000059      1.0  3.324723e-06  3.324723e-06   
F:B:C                 0.000059      1.0  3.324723e-06  3.324723e-06   
F:I:C                 0.000059      1.0  3.324723e-06  3.324723e-06   
B:I:C                 0.003581      1.0  6.206353e-03  6.206353e-03   
F:B:I:C              -0.000059      1.0  3.324723e-06  3.324723e-06   
Residual/Intercept    0.495972    784.0  2.857390e-27  3.644630e-30   

                                                    Weighted F1                \
                               F PR(>F) Coefficient          df        sum_sq   
F                   9.122251e+23    0.0    0.000064         1.0  2.260818e-06   
B                   2.250602e+27    0.0   -0.003202         1.0  1.010310e-03   
I                   2.659061e+26    0.0   -0.001101         1.0  2.747927e-03   
C                   6.329024e+27    0.0    0.005370         1.0  6.329243e-02   
F:B                 9.122251e+23    0.0   -0.000064         1.0  2.260818e-06   
F:I                 9.122251e+23    0.0   -0.000064         1.0  2.260818e-06   
F:C                 9.122251e+23    0.0   -0.000064         1.0  2.260818e-06   
B:I                 1.352353e+25    0.0    0.000248         1.0  4.508266e-03   
B:C                 5.986871e+23    0.0   -0.000052         1.0  2.152654e-07   
I:C                 2.635640e+27    0.0   -0.003465         1.0  1.352137e-02   
F:B:I               9.122251e+23    0.0    0.000064         1.0  2.260818e-06   
F:B:C               9.122251e+23    0.0    0.000064         1.0  2.260818e-06   
F:I:C               9.122251e+23    0.0    0.000064         1.0  2.260818e-06   
B:I:C               1.702876e+27    0.0    0.002785         1.0  1.221702e-02   
F:B